In [12]:
!pip install numpy

In [1]:
import pandas as pd
import os
import json
import numpy as np
import matplotlib.pyplot as plt

In [2]:
folder = "/media3/CRP8/TDE/data/DR_photometry_full/extragalactic"

In [3]:
list_files = os.listdir(folder)

In [4]:
df = pd.read_csv(os.path.join(folder, list_files[0]))

In [5]:
df.catflags.value_counts()

catflags
0        2291
32768     188
33024       2
1           1
Name: count, dtype: int64

In [6]:
band_colors = {'g': 'green', 'r': 'blue', 'i': 'gold'}

In [7]:
df["filtercode"]

0       zg
1       zg
2       zg
3       zg
4       zg
        ..
2477    zr
2478    zr
2479    zr
2480    zr
2481    zr
Name: filtercode, Length: 2482, dtype: object

In [ ]:
csv_files = [f for f in os.listdir(folder) if f.endswith('.csv')]
band_colors = {'zg': 'green', 'zr': 'blue', 'zi': 'gold'}

outdir = "../../img/extragalactic_catflags"
os.makedirs(outdir, exist_ok=True)

for file in csv_files:
    df = pd.read_csv(os.path.join(folder, file))
    required = {'catflags', 'mjd', 'mag', 'magerr', 'filtercode'}
    if not required.issubset(df.columns):
        print(f"Skipping {file}: missing columns")
        continue

    # Filter good and bad flags
    df_good = df[df['catflags'] == 0]
    df_bad = df[df['catflags'] != 0]

    plt.figure(figsize=(8, 5))
    # Plot each filtercode separately for good flags
    for b in np.unique(df_good['filtercode']):
        mask = df_good['filtercode'] == b
        plt.errorbar(
            df_good['mjd'][mask], df_good['mag'][mask], yerr=df_good['magerr'][mask],
            fmt='o', ms=4, color=band_colors.get(b, 'black'), markeredgecolor='black', label=f"{b}-band"
        )
    # Plot bad flags as red stars
    if not df_bad.empty:
        plt.scatter(df_bad['mjd'], df_bad['mag'], color='red', marker='*', s=80, label='catflags!=0')
    plt.gca().invert_yaxis()
    plt.xlabel("MJD")
    plt.ylabel("Magnitude")
    plt.title(f"Light Curve for {file}")
    plt.suptitle(f"bad quality: {len(df_bad)} / total: {len(df)}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"{file.replace('.csv', '')}.png"))
    plt.close()

/tmp/ipykernel_2300682/1139913072.py:33: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
